# Single-Step Tool Use

<a target="_blank" href="https://colab.research.google.com/github/cohere-ai/notebooks/blob/main/notebooks/llmu/single_step_tool_use.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

*Read the accompanying [article here](https://cohere.ai/blog/single-step-tool-use/).*

With Cohere, you can tool use in two different modes: single-step and multi-step.

In a single-step tool use scenario, an LLM calls a single tool or many tools in parallel. The tools then return the results, and the LLM uses the results to generate its response.

In this notebook, we look at single-step tool use and see how tool use works in the following scenarios:
-  Multiple tools: What if the model had multiple tools at its disposal? How does the model reason over which tools to use at a particular time?
- Parallel tool calls: What does it look like if more than one tool call is required?
- When not to use tools: What if a question can, and should, be answered directly by a model without needing a tool?
- Tool use in a chat setting: What if the user had follow-up questions, i.e., how does an assistant maintain the context of a conversation in a multi-turn chat setting?

First, let’s install the Cohere Python SDK and set up the Cohere client.

In [ ]:
! pip install cohere -q

In [40]:
import cohere
import json
co = cohere.Client("COHERE_API_KEY") # Get your API key: https://dashboard.cohere.com/api-keys

# Single step, parallel tool use

Let’s now create two tools:
- A function to query the sales database called `query_daily_sales_report`, which includes a mock sales database
- A function to query the product catalog called `query_product_catalog`, which includes a mock product catalog

In [41]:
def daily_sales_report(day: str) -> dict:
    """
    Function to retrieve the sales report for the given day
    """
    # Mock database containing daily sales reports
    sales_database = {
    '2023-09-28': {'total_sales_amount': 5000,'total_units_sold': 100},
    '2023-09-29': {'total_sales_amount': 10000,'total_units_sold': 250},
    '2023-09-30': {'total_sales_amount': 8000,'total_units_sold': 200}
    }
    
    report = sales_database.get(day, {})
    
    if report:
        return {
            'date': day,
            'summary': f"Total Sales Amount: {report['total_sales_amount']}, Total Units Sold: {report['total_units_sold']}"
        }
    else:
        return {'date': day, 'summary': 'No sales data available for this day.'}
    

def product_database(category: str) -> dict:
    """
    Function to retrieve products for the given category
    """
    
    # Mock product catalog
    product_catalog = {
        'Electronics': [
            {'product_id': 'E1001', 'name': 'Smartphone', 'price': 500, 'stock_level': 20},
            {'product_id': 'E1002', 'name': 'Laptop', 'price': 1000, 'stock_level': 15},
            {'product_id': 'E1003', 'name': 'Tablet', 'price': 300, 'stock_level': 25},
        ],
        'Clothing': [
            {'product_id': 'C1001', 'name': 'T-Shirt', 'price': 20, 'stock_level': 100},
            {'product_id': 'C1002', 'name': 'Jeans', 'price': 50, 'stock_level': 80},
            {'product_id': 'C1003', 'name': 'Jacket', 'price': 100, 'stock_level': 40},
        ]
    }
    
    products = product_catalog.get(category, [])
    return {
        'category': category,
        'products': products
    }


functions_map = {
    "daily_sales_report": daily_sales_report,
    "product_database": product_database
}

Next, we define the tool schema for the two tools. Both functions accept one parameter, `day` and `category` respectively.

In [42]:
tools = [
    {
        "name": "daily_sales_report",
        "description": "Connects to a database to retrieve overall sales volumes and sales information for a given day.",
        "parameter_definitions": {
            "day": {
                "description": "Retrieves sales data for this day, formatted as YYYY-MM-DD.",
                "type": "str",
                "required": True
            }
        }
    },
    {
        "name": "product_database",
        "description": "A database that contains information about all the products of this company, including categories, prices, and stock levels.",
        "parameter_definitions": {
            "category": {
                "description": "Retrieves product information data for all products in this category.",
                "type": "str",
                "required": True
            }
        }
    }
]

Next, we define a preamble (optional).

In [43]:
preamble = """## Task & Context
You are an assistant for an e-commerce company. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a set of tools, which you use to get your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging.

## Style Guide
Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling.
"""

# Create a function to run the assistant

Let's create a function called `run_assistant` which does the following:
- Gets the user message (Step 1)
- Calls the Chat endpoint for tool call generation (Step 2)
- If the response contains at least one tool call, executes the tool call(s) and gets the tool results (Step 3)
- Generates the final response with citations (Step 4)

We put Steps 2 and 3 in a loop, checking if the model’s response contains tool calls. This allows the assistant to handle any number of tool call steps (zero, one, or multiple) for a given user message.

We are also enabling the assistant to retain the state (or memory) of a conversation, which allows it to handle multi-turn chat scenarios. We do this by having the run_assistant function accept the chat history from the previous turns as an argument and return the updated chat history.


In [44]:
model = "command-r-plus"

def run_assistant(message, chat_history=[]):
    # Step 1: Get user message
    print(f"Question:\n{message}")
    print("="*50)

    # Step 2: Generate tool calls (if any)    
    response = co.chat(
        message=message,
        model=model,
        preamble=preamble,
        tools=tools,
        chat_history=chat_history,
        force_single_step=True
    )

    while response.tool_calls:
        tool_calls = response.tool_calls
        
        if response.text:
            print("Intermediate response:")
            print(response.text,"\n")
        print("Tool calls:")
        for call in tool_calls:
            print(f"Tool name: {call.name} | Parameters: {call.parameters}")
        print("="*50)
        
        # Step 3: Get tool results
        tool_results = []
        for tc in tool_calls:
            tool_call = {"name": tc.name, "parameters": tc.parameters}
            tool_output = functions_map[tc.name](**tc.parameters)
            tool_results.append({"call": tool_call, "outputs": [tool_output]})
        
        # Step 4: Generate response and citations                
        response = co.chat(
            message="",
            model=model,
            preamble=preamble,
            tools=tools,
            tool_results=tool_results,
            chat_history=response.chat_history
        )

        # Append the current chat turn to the chat history
        chat_history = response.chat_history
        
    # Print final response
    print("Final response:")
    print(response.text)
    print("="*50)
    
    # Print citations (if any)
    if response.citations:
        print("Citations:")
        for citation in response.citations:
            print(citation)
        print("\nCited Documents:")
        for document in response.documents:
            print(document)
        print("="*50)
    
    return chat_history

## Single-step

Let’s now ask the first question to the assistant with a simple example.

The assistant correctly identifies that, out of the two tools available, `query_daily_sales_report` is sufficient to answer the question. And it answers the question correctly.

In [45]:
chat_history = run_assistant("Can you provide a sales summary for 29th September 2023?")

Question:
Can you provide a sales summary for 29th September 2023?
Tool calls:
Tool name: daily_sales_report | Parameters: {'day': '2023-09-29'}
Final response:
On 29 September 2023, we sold 250 units, totalling a sales amount of 10,000.
Citations:
start=30 end=39 text='250 units' document_ids=['daily_sales_report:0:2:0']
start=53 end=75 text='sales amount of 10,000' document_ids=['daily_sales_report:0:2:0']

Cited Documents:
{'date': '2023-09-29', 'id': 'daily_sales_report:0:2:0', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250', 'tool_name': 'daily_sales_report'}


In [46]:
# Print chat history
for turn in chat_history:
    print(turn,"\n")

message='Can you provide a sales summary for 29th September 2023?' tool_calls=None role='USER' 

message=None tool_calls=[ToolCall(name='daily_sales_report', parameters={'day': '2023-09-29'})] role='CHATBOT' 

tool_results=[ToolResult(call=ToolCall(name='daily_sales_report', parameters={'day': '2023-09-29'}), outputs=[{'date': '2023-09-29', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250'}])] role='TOOL' 

message='On 29 September 2023, we sold 250 units, totalling a sales amount of 10,000.' tool_calls=None role='CHATBOT' 



## Single-step, parallel

The model can decide that more than one tool is required to provide a response. This means calling multiple tools in parallel within the same step. This can either be:
- Calling different tools in parallel
- Calling the same tool multiple times in parallel
- A combination of both

To illustrate this, let’s try to ask the model another question. Let’s ask about the sales summary of two different dates and also about the stock level information of a product category.

This time, the model generates three tool calls in parallel:
- Two queries to the sales database, one for each date
- One query to the product catalog

These are exactly what are needed to answer the question. And it correctly answers the question.


In [31]:
chat_history = run_assistant("Can you provide a sales summary for 28th and 29th September 2023 as well as the stock level of the products in the 'Electronics' category?")

Question:
Can you provide a sales summary for 28th and 29th September 2023 as well as the stock level of the products in the 'Electronics' category?
Tool calls:
Tool name: daily_sales_report | Parameters: {'day': '2023-09-28'}
Tool name: daily_sales_report | Parameters: {'day': '2023-09-29'}
Tool name: product_database | Parameters: {'category': 'Electronics'}
Final response:
On 28 September 2023, the total sales amount was 5000, with 100 units sold. The following day, 29 September 2023, the total sales amount was 10000, with 250 units sold. 

Here is the stock level of the products in the 'Electronics' category:
- Smartphone (E1001) - 20
- Laptop (E1002) - 15
- Tablet (E1003) - 25
Citations:
start=3 end=20 text='28 September 2023' document_ids=['daily_sales_report:0:2:0']
start=26 end=53 text='total sales amount was 5000' document_ids=['daily_sales_report:0:2:0']
start=60 end=74 text='100 units sold' document_ids=['daily_sales_report:0:2:0']
start=95 end=112 text='29 September 2023' d

## Directly answering

A key attribute of tool use systems is the model’s ability to choose the right tools for a task. And that also includes the ability to decide to not use any tool, and instead, to respond to a user message directly.

### Questions that don't require tools

Let’s look at the first scenario. The question below asks about building a great company. This is a rather general question that a good LLM would be able to answer directly without needing any additional help.

Notice that the model doesn’t trigger any tool calls, but instead goes directly to answering the question.

In [33]:
chat_history = run_assistant("Give me 3 concise tips on how to build a great company")

Question:
Give me 3 concise tips on how to build a great company
Final response:
1. Start with a strong foundation: Build a solid business plan, secure adequate funding, and establish a clear mission and vision. 

2. Focus on people: Hire and retain talented employees by creating a positive company culture and offering competitive benefits. 

3. Stay agile: Adapt to market changes and be open to innovation. Stay connected to your customers' needs and wants and be willing to pivot when necessary.


### Questions that require tools but tools not available

Let’s look at the other scenario. The question below is about the company's employee count, which is a specific piece of information that an LLM would not possess and requires context from an external tool.

But because none of the available tools can provide this information, the model doesn’t attempt any tool call. Instead, it responds directly to the user question mentioning that it doesn’t have the information needed to answer the question.

In [34]:
chat_history = run_assistant("How many employees does this company have?")

Question:
How many employees does this company have?
Final response:
I don't have any information about the company's number of employees. Is there anything else I can help you with?


## State management (memory)

The `run_assistant` function is already enabled to handle multi-turn chat scenarios. The function represents one turn of a conversation. For each turn, it accepts the latest chat history of a conversation and returns the updated one once the turn has been completed.

This can continue for any number of turns, the limit being the model’s maximum context length (with Command R/R+, that’s 128k tokens).

The chat history for each turn consists of:
- The USER message
- Followed by the CHATBOT message with the list of tool calls
- Followed the TOOL message with the list of tool results
- Finally, followed by the CHATBOT message with the final response to the user

Let’s try it out, starting with the same question about the sales summary.


In [35]:
chat_history = run_assistant("Can you provide a sales summary for 29th September 2023?")

Question:
Can you provide a sales summary for 29th September 2023?
Tool calls:
Tool name: daily_sales_report | Parameters: {'day': '2023-09-29'}
Final response:
On 29 September 2023, the total sales amount was 10,000 and the total number of units sold was 250.
Citations:
start=26 end=55 text='total sales amount was 10,000' document_ids=['daily_sales_report:0:2:0']
start=64 end=98 text='total number of units sold was 250' document_ids=['daily_sales_report:0:2:0']

Cited Documents:
{'date': '2023-09-29', 'id': 'daily_sales_report:0:2:0', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250', 'tool_name': 'daily_sales_report'}


Let’s now ask a follow-up question, a rather vague question that a model would not be able to answer without the context of the previous turn. Here we pass the first turn’s chat history to the `run_assistant` function.

The model is able to infer that “the 28th” is likely referring to September 28th 2023, because of what was asked in the previous turn. 

In [36]:
chat_history = run_assistant("What about the 28th?", chat_history)

Question:
What about the 28th?
Tool calls:
Tool name: daily_sales_report | Parameters: {'day': '2023-09-28'}
Final response:
On 28 September 2023, the total sales amount was 5,000 and the total number of units sold was 100.
Citations:
start=26 end=54 text='total sales amount was 5,000' document_ids=['daily_sales_report:0:6:0']
start=63 end=97 text='total number of units sold was 100' document_ids=['daily_sales_report:0:6:0']

Cited Documents:
{'date': '2023-09-28', 'id': 'daily_sales_report:0:6:0', 'summary': 'Total Sales Amount: 5000, Total Units Sold: 100', 'tool_name': 'daily_sales_report'}


Let’s continue the conversation.

Again, the model is able to infer what “both days” are from the chat context.

In [37]:
chat_history = run_assistant("How many units were sold over both days", chat_history)

Question:
How many units were sold over both days
Tool calls:
Tool name: daily_sales_report | Parameters: {'day': '2023-09-29'}
Tool name: daily_sales_report | Parameters: {'day': '2023-09-28'}
Final response:
Combined, 28 and 29 September 2023 saw 350 units sold.
Citations:
start=39 end=53 text='350 units sold' document_ids=['daily_sales_report:0:10:0', 'daily_sales_report:1:10:0']

Cited Documents:
{'date': '2023-09-29', 'id': 'daily_sales_report:0:10:0', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250', 'tool_name': 'daily_sales_report'}
{'date': '2023-09-28', 'id': 'daily_sales_report:1:10:0', 'summary': 'Total Sales Amount: 5000, Total Units Sold: 100', 'tool_name': 'daily_sales_report'}


Here’s a look at the chat history. It consists of a set of USER, CHATBOT, and TOOL messages appended in the right sequence of turns, providing the model with the right context each time it generates a new response.

In [39]:
# Print chat history
for turn in chat_history:
    print(turn,"\n")

message='Can you provide a sales summary for 29th September 2023?' tool_calls=None role='USER' 

message=None tool_calls=[ToolCall(name='daily_sales_report', parameters={'day': '2023-09-29'})] role='CHATBOT' 

tool_results=[ToolResult(call=ToolCall(name='daily_sales_report', parameters={'day': '2023-09-29'}), outputs=[{'date': '2023-09-29', 'summary': 'Total Sales Amount: 10000, Total Units Sold: 250'}])] role='TOOL' 

message='On 29 September 2023, the total sales amount was 10,000 and the total number of units sold was 250.' tool_calls=None role='CHATBOT' 

message='What about the 28th?' tool_calls=None role='USER' 

message=None tool_calls=[ToolCall(name='daily_sales_report', parameters={'day': '2023-09-28'})] role='CHATBOT' 

tool_results=[ToolResult(call=ToolCall(name='daily_sales_report', parameters={'day': '2023-09-28'}), outputs=[{'date': '2023-09-28', 'summary': 'Total Sales Amount: 5000, Total Units Sold: 100'}])] role='TOOL' 

message='On 28 September 2023, the total sales a